### Import Historical Data by IBK API

Import data from 01. 01. 2016 to 01. 11. 2025

In [ ]:
import datetime
from ib_insync import *
import nest_asyncio


#Connect to IBK
nest_asyncio.apply()
ib = IB()
ib.connect("127.0.0.1", 7497, clientId=1)

#Define Stock Contract
contract = Stock("QQQ", "SMART", "USD")

#Download historical data
dt = "" #For datetime, an empty string is interpreted as "now".
barsList = []

while True:
    bars = ib.reqHistoricalData(contract, endDateTime = dt, durationStr="20 D", barSizeSetting= "5 mins", whatToShow= "TRADES", useRTH= True, formatDate =1)

    if not bars:
        break

    barsList.append(bars)
    dt = bars[0].date  # This step is crucial for the loop to work:
                   # The first time, dt = "" → meaning "up to now". 
                   # After retrieving a block of 20 days, dt is set to bars[0].date, 
                   # i.e., the date of the first bar in the block just received.
                   # On the next iteration, we retrieve the 20 days preceding the first bar of the last received block.
    print("Retrieved data up to:", dt)


    if dt.date() < datetime.date(2015, 1, 1):
    # This sets the lower limit for the data we want to retrieve.
    # Note: IB downloads data in blocks, so if the first bar of the last block
    # is still within the range, it will also download the next block.
        break

#Save data to CSV
allBars = []
for bars in reversed(barsList):   # Loop through each block of bars. reversed() is used because the original order is from newest to oldest.
    for b in bars:                # Loop through each bar within the block
        allBars.append(b)         # Append the single bar to the final list
df = util.df(allBars)             # Convert the list of bars to a DataFrame
df.to_csv(f"{contract.symbol}_historical_data.csv", index = False)
print("Historical data saved")
ib.disconnect()

Retrieved data up to: 2025-11-26 09:30:00-05:00
Retrieved data up to: 2025-10-29 09:30:00-04:00
Retrieved data up to: 2025-10-01 09:30:00-04:00
Retrieved data up to: 2025-09-03 09:30:00-04:00
Retrieved data up to: 2025-08-05 09:30:00-04:00
Retrieved data up to: 2025-07-08 09:30:00-04:00
Retrieved data up to: 2025-06-06 09:30:00-04:00
Retrieved data up to: 2025-05-08 09:30:00-04:00
Retrieved data up to: 2025-04-09 09:30:00-04:00
Retrieved data up to: 2025-03-12 09:30:00-04:00
Retrieved data up to: 2025-02-11 09:30:00-05:00
Retrieved data up to: 2025-01-13 09:30:00-05:00
Retrieved data up to: 2024-12-11 09:30:00-05:00
Retrieved data up to: 2024-11-12 09:30:00-05:00
Retrieved data up to: 2024-10-15 09:30:00-04:00
Retrieved data up to: 2024-09-17 09:30:00-04:00
Retrieved data up to: 2024-08-19 09:30:00-04:00
Retrieved data up to: 2024-07-22 09:30:00-04:00
Retrieved data up to: 2024-06-21 09:30:00-04:00
Retrieved data up to: 2024-05-22 09:30:00-04:00
Retrieved data up to: 2024-04-24 09:30:0

### SQL Data Cleaning

```sql
create table qqq_historical_data (date timestamp, open float, high float, low float, close float, volume int, average float, barcount int);

SET GLOBAL local_infile=1;

load data local infile 'C:/Users/nicho/Desktop/Trading Space/Progetto QQQ/QQQ_historical_data.csv' into table qqq_historical_data
fields terminated by ','
lines terminated by '\n'
ignore 1 lines
(date ,open ,high ,low ,close ,volume ,average ,barCount);

alter table qqq_historical_data add column date_ny datetime;

set sql_safe_updates = 0;

update qqq_historical_data set date_ny = convert_tz(date, 'UTC', 'America/New_York');

alter table qqq_historical_data drop column date, drop column volume, drop column average, drop column barcount;

create table qqq_historical_data1 select date_ny, open, high, low, close from qqq_historical_data where date(date_ny) >= '2015-01-01' and date(date_ny) <= '2025-12-01';

WITH
list_open as (select date(date_ny) as date, min(time(date_ny)) as open_time from qqq_historical_data1 group by date(date_ny)),
list_day_open as (select * from list_open where open_time != '09:30:00'),
list_close as (select date(date_ny) as date, max(time(date_ny)) as close_time from qqq_historical_data1 group by date(date_ny)),
list_day_close as (select * from list_close where close_time != '15:55:00'),
list_day as (select date, open_time, NULL AS close_time from list_day_open union all select date, NULL AS open_time, close_time from list_day_close)
delete from qqq_historical_data1 where date(date_ny) in (select date from list_day);

WITH
list_open as (select date(date_ny) as date, min(time(date_ny)) as open_time from qqq_historical_data1 group by date(date_ny)),
list_day_open as (select * from list_open where open_time != '09:30:00'),
list_close as (select date(date_ny) as date, max(time(date_ny)) as close_time from qqq_historical_data1 group by date(date_ny)),
list_day_close as (select * from list_close where close_time != '15:55:00'),
list_day as (select date, open_time, NULL AS close_time from list_day_open union all select date, NULL AS open_time, close_time from list_day_close)
select * from list_day;

```

### Environment Setup

In [ ]:
from tqdm import tqdm
import datetime as dt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import mysql.connector
password_sql = 'YOUR_PASSWORD'

qqq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "qqq")

### Define Strategy Classes

In [4]:
class trading_strategy:
    def __init__(self):
        self.initial_account = 0
        self.account = [self.initial_account]
        self.buy_pos = 0
        self.sell_pos = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.trades_pnl = []
        self.fee = 0
        self.r = 0
        self.r_history = []
        self.opening_price = 0
        self.sell_price = 0
        self.sell_price_history = []
        self.buy_price = 0
        self.buy_price_history = []
        self.dates = []
        self.unit_trades = []
        self.unit_list = []
        

    def record_trade(self, date):
        self.units = 1
        self.unit_list.append(self.units)
        self.trades_pnl.append((self.sell_price - self.buy_price)*self.units - 2*self.units*self.fee)
        self.account.append(self.account[-1] + self.trades_pnl[-1])
        self.dates.append(date)
        self.buy_price_history.append(self.buy_price)
        self.sell_price_history.append(self.sell_price)
        self.r = ((self.sell_price - self.buy_price)/(abs(self.opening_price - self.stop_loss)))
        self.r_history.append(self.r)
        self.unit_trades.append(self.trades_pnl[-1]/self.units)
    
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0
        self.opening_price = 0
        

In [5]:
class trading_strategy_sim:
    def __init__(self):
        self.initial_account = 25000
        self.account = [self.initial_account]
        self.buy_pos = 0
        self.sell_pos = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.trades_pnl = []
        self.fee = 0.0005
        self.r = 0
        self.r_history = []
        self.opening_price = 0
        self.sell_price = 0
        self.sell_price_history = []
        self.buy_price = 0
        self.buy_price_history = []
        self.dates = []
        self.unit_trades = []
        self.unit_list = []
        self.risk = 0

    def record_trade(self, date):
        self.units = int(min((self.account[-1]*0.01)/self.risk, (4*self.account[-1])/self.opening_price))
        self.unit_list.append(self.units)
        self.trades_pnl.append((self.sell_price - self.buy_price)*self.units - 2*self.units*self.fee)
        self.account.append(self.account[-1] + self.trades_pnl[-1])
        self.dates.append(date)
        self.buy_price_history.append(self.buy_price)
        self.sell_price_history.append(self.sell_price)
        self.r = ((self.sell_price - self.buy_price)/(abs(self.opening_price - self.stop_loss)))
        self.r_history.append(self.r)
        self.unit_trades.append(self.trades_pnl[-1]/self.units)
    
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0
        self.opening_price = 0
        self.risk = 0

    def restart(self):
        self.__init__()

### Data for Strategies

In [6]:
qqq_historical_data = pd.read_sql('Select * From qqq_historical_data1 where date(date_ny) >= "2016-01-01"', qqq_connection_sql)
print(qqq_historical_data.dtypes)

qqq_historical_data['date'] = qqq_historical_data['date_ny'].dt.date
qqq_historical_data['time'] = qqq_historical_data['date_ny'].dt.time

qqq_historical_data = qqq_historical_data.sort_values(by=["date", "time"]).reset_index(drop=True)


C:\Users\nicho\AppData\Local\Temp\ipykernel_7732\3282340993.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



date_ny    datetime64[ns]
open              float64
high              float64
low               float64
close             float64
dtype: object


### Buy & Hold

In [7]:
buy_hold_open = qqq_historical_data['open'].iloc[0]
buy_hold_close = qqq_historical_data.groupby(qqq_historical_data['date'])['close'].last()
buy_hold_account = buy_hold_close - buy_hold_open

### Buy & Hold Intraday

In [8]:
buy_hold_intraday_open = qqq_historical_data.groupby(qqq_historical_data['date'])['open'].first()
buy_hold_intraday_close = qqq_historical_data.groupby(qqq_historical_data['date'])['close'].last()
buy_hold_intraday_pnl = buy_hold_intraday_close - buy_hold_intraday_open
buy_hold_intraday_account = buy_hold_intraday_pnl.cumsum()


### Base Strategy 

```sql
create table base_strategy
WITH
direction as (select date(date_ny) as date, case when (close - open) > 0 then 'Buy' when (close - open) < 0 then 'Sell' end as direction from qqq_historical_data1 where time(date_ny) = '09:30:00'),
entry_price as (select date(date_ny) as date, open as entry_price from qqq_historical_data1 where time(date_ny) = '09:35:00'),
close_price as (select date(date_ny) as date, close as close_price from qqq_historical_data1 where time(date_ny) = '15:55:00'),
base_strategy as (select direction.*, entry_price.entry_price, close_price.close_price from direction join entry_price on direction.date = entry_price.date join close_price on direction.date = close_price.date)
select * from base_strategy
```

In [9]:
base_strategy_db = pd.read_sql('Select * from base_strategy', qqq_connection_sql)
print(base_strategy_db.dtypes)

basic_strategy_db = base_strategy_db.sort_values(by=["date"]).reset_index(drop=True)

date            object
direction       object
entry_price    float64
close_price    float64
dtype: object


C:\Users\nicho\AppData\Local\Temp\ipykernel_7732\2917463922.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [10]:
base_strategy_pnl = []
base_strategy_dates = []
for x in range(0,len(base_strategy_db)):
    if base_strategy_db['direction'].iloc[x] == 'Buy':
        base_strategy_realized = base_strategy_db['close_price'].iloc[x] - base_strategy_db['entry_price'].iloc[x]
        base_strategy_pnl.append(base_strategy_realized)
        base_strategy_dates.append(base_strategy_db['date'].iloc[x])
    elif base_strategy_db['direction'].iloc[x] == 'Sell':
        base_strategy_realized = base_strategy_db['entry_price'].iloc[x] - base_strategy_db['close_price'].iloc[x]
        base_strategy_pnl.append(base_strategy_realized)
        base_strategy_dates.append(base_strategy_db['date'].iloc[x])
base_strategy_account = np.cumsum(base_strategy_pnl)

### Basic Strategy VS Buy & Hold VS Buy & Hold Intraday

In [11]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = base_strategy_dates, y = base_strategy_account, mode = 'lines', name = 'Basic Strategy'))
fig1.add_trace(go.Scatter(x = buy_hold_account.index, y = buy_hold_account.values, mode = 'lines', name = 'Buy & Hold'))
fig1.add_trace(go.Scatter(x = buy_hold_intraday_account.index, y = buy_hold_intraday_account.values, mode = 'lines', name = 'Buy & Hold Intraday'))
fig1.update_layout(title = dict(text = "Basic Strategy VS Buy & Hold VS Buy & Hold Intraday", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### Test 1

```sql
create temporary table target_list
WITH
sl_list as (select date(date_ny) as date, case when direction = 'Buy' then low when direction = 'Sell' then high end as stop_loss from (select qqq_historical_data1.*, base_strategy.* from qqq_historical_data1 join base_strategy on date(qqq_historical_data1.date_ny) = base_strategy.date) as full_table where time(date_ny) = '09:30:00'),
risk_list as (select *, round(entry_price - stop_loss,2) as risk from (select base_strategy.*, sl_list.stop_loss from base_strategy join sl_list on base_strategy.date = sl_list.date) as work1),
target_list as (select date, direction, entry_price, close_price, stop_loss, round((entry_price + risk*10),2) as target from risk_list)
select * from target_list;
```
---
```sql
create table test1_data select qqq_historical_data1.*, target_list.direction, target_list.stop_loss, target_list.target from qqq_historical_data1 join target_list on date(qqq_historical_data1.date_ny) = target_list.date
```

In [12]:
test1_db = pd.read_sql('Select * from test1_data', qqq_connection_sql)
print(test1_db.dtypes)

test1_db['date'] = test1_db['date_ny'].dt.date
test1_db['time'] = test1_db['date_ny'].dt.time

test1_db = test1_db.sort_values(by=["date", "time"]).reset_index(drop=True)

C:\Users\nicho\AppData\Local\Temp\ipykernel_7732\776540865.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



date_ny      datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
direction            object
stop_loss           float64
target              float64
dtype: object


In [13]:
test1 = trading_strategy()

In [14]:
for x in tqdm(range(0, len(test1_db))):

    # Entry Logic
    if test1_db['time'].iloc[x] == dt.time(9,35,0):
        if test1_db['direction'].iloc[x] == 'Buy':
            test1.buy_pos = 1
            test1.buy_price = test1_db['open'].iloc[x]
            test1.stop_loss = test1_db['stop_loss'].iloc[x]
            test1.take_profit = test1_db['target'].iloc[x]
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'BUY at:', test1.buy_price,
                  'SL at:', test1.stop_loss, 'TP at:', test1.take_profit)

        if test1_db['direction'].iloc[x] == 'Sell':
            test1.sell_pos = 1
            test1.sell_price = test1_db['open'].iloc[x]
            test1.stop_loss = test1_db['stop_loss'].iloc[x]
            test1.take_profit = test1_db['target'].iloc[x]
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'SELL at:', test1.sell_price,
                  'SL at:', test1.stop_loss, 'TP at:', test1.take_profit)

    # Long Position Management 
    if test1.buy_pos == 1 and test1.sell_pos == 0:
        if test1_db['low'].iloc[x] <= test1.stop_loss:
            test1.sell_pos = 1
            test1.sell_price = test1.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL STOP at:', test1.sell_price, '[close]')
        elif test1_db['high'].iloc[x] >= test1.take_profit:
            test1.sell_pos = 1
            test1.sell_price = test1.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL TP at:', test1.sell_price, '[close]')

    # Short Position Management
    if test1.buy_pos == 0 and test1.sell_pos == 1:
        if test1_db['high'].iloc[x] >= test1.stop_loss:
            test1.buy_pos = 1
            test1.buy_price = test1.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY STOP at:', test1.buy_price, '[close]')
        elif test1_db['low'].iloc[x] <= test1.take_profit:
            test1.buy_pos = 1
            test1.buy_price = test1.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY TP at:', test1.buy_price, '[close]')
            
    # End Day
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if test1.buy_pos == 1 and test1.sell_pos == 0:
            test1.sell_pos = 1
            test1.sell_price = test1_db['close'].iloc[x]
            print('SELL at:', test1_db['close'].iloc[x], '[close]')

        if test1.buy_pos == 0 and test1.sell_pos == 1:
            test1.buy_pos = 1
            test1.buy_price = test1_db['close'].iloc[x]
            print('BUY at:', test1_db['close'].iloc[x], '[close]')

    #Trade Record
    if test1.buy_pos == 1 and test1.sell_pos == 1:
        test1.record_trade(test1_db['date'].iloc[x])
        test1.reset()



  4%|▍         | 7300/191412 [00:00<00:05, 36710.39it/s]

2016-01-04 09:35:00 SELL at: 109.13 SL at: 109.45 TP at: 105.93
2016-01-04 15:55:00 BUY STOP at: 109.45 [close]
2016-01-05 09:35:00 SELL at: 109.85 SL at: 110.18 TP at: 106.55
BUY at: 109.24 [close]
2016-01-06 09:35:00 BUY at: 107.57 SL at: 107.41 TP at: 109.17
SELL at: 108.27 [close]
2016-01-07 09:35:00 SELL at: 105.8 SL at: 106.0 TP at: 103.8
2016-01-07 09:35:00 BUY STOP at: 106.0 [close]
2016-01-08 09:35:00 BUY at: 106.06 SL at: 105.53 TP at: 111.36
2016-01-08 10:10:00 SELL STOP at: 105.53 [close]
2016-01-11 09:35:00 BUY at: 104.88 SL at: 104.73 TP at: 106.38
2016-01-11 09:35:00 SELL STOP at: 104.73 [close]
2016-01-12 09:35:00 BUY at: 105.6 SL at: 105.24 TP at: 109.2
2016-01-12 10:45:00 SELL STOP at: 105.24 [close]
2016-01-13 09:35:00 SELL at: 105.97 SL at: 106.12 TP at: 104.47
2016-01-13 09:35:00 BUY STOP at: 106.12 [close]
2016-01-14 09:35:00 SELL at: 102.17 SL at: 102.44 TP at: 99.47
2016-01-14 10:25:00 BUY STOP at: 102.44 [close]
2016-01-15 09:35:00 BUY at: 101.02 SL at: 100.44 

  6%|▌         | 10972/191412 [00:00<00:05, 35032.70it/s]

SELL at: 108.45 [close]
2016-05-25 09:35:00 BUY at: 108.88 SL at: 108.83 TP at: 109.38
2016-05-25 09:35:00 SELL STOP at: 108.83 [close]
2016-05-26 09:35:00 BUY at: 109.4 SL at: 109.26 TP at: 110.8
2016-05-26 09:35:00 SELL STOP at: 109.26 [close]
2016-05-27 09:35:00 BUY at: 109.72 SL at: 109.61 TP at: 110.82
SELL at: 110.12 [close]
2016-05-31 09:35:00 SELL at: 110.23 SL at: 110.3 TP at: 109.53
2016-05-31 09:35:00 BUY STOP at: 110.3 [close]
2016-06-01 09:35:00 BUY at: 110.06 SL at: 109.99 TP at: 110.76
2016-06-01 09:35:00 SELL STOP at: 109.99 [close]
2016-06-02 09:35:00 SELL at: 110.08 SL at: 110.16 TP at: 109.28
2016-06-02 10:35:00 BUY STOP at: 110.16 [close]
2016-06-03 09:35:00 SELL at: 110.29 SL at: 110.42 TP at: 108.99
BUY at: 110.06 [close]
2016-06-06 09:35:00 SELL at: 110.19 SL at: 110.26 TP at: 109.49
2016-06-06 09:35:00 BUY STOP at: 110.26 [close]
2016-06-07 09:35:00 SELL at: 110.5 SL at: 110.54 TP at: 110.1
2016-06-07 09:45:00 BUY STOP at: 110.54 [close]
2016-06-08 09:35:00 SELL

 11%|█         | 21383/191412 [00:00<00:05, 31243.97it/s]

...

2024-03-25 09:35:00 BUY at: 443.87 SL at: 443.13 TP at: 451.27
2024-03-25 09:40:00 SELL STOP at: 443.13 [close]
2024-03-26 09:35:00 BUY at: 446.76 SL at: 445.85 TP at: 455.86
2024-03-26 10:25:00 SELL STOP at: 445.85 [close]
2024-03-27 09:35:00 SELL at: 445.31 SL at: 446.54 TP at: 433.01
BUY at: 444.94 [close]
2024-03-28 09:35:00 SELL at: 444.24 SL at: 444.9 TP at: 437.64
2024-03-28 09:40:00 BUY STOP at: 444.9 [close]
2024-04-01 09:35:00 BUY at: 446.16 SL at: 444.55 TP at: 462.26
2024-04-01 10:35:00 SELL STOP at: 444.55 [close]
2024-04-02 09:35:00 SELL at: 439.61 SL at: 440.19 TP at: 433.81
2024-04-02 13:00:00 BUY STOP at: 440.19 [close]
2024-04-03 09:35:00 BUY at: 439.7 SL at: 438.95 TP at: 447.2
SELL at: 442.06 [close]
2024-04-04 09:35:00 BUY at: 446.54 SL at: 446.05 TP at: 451.44
2024-04-04 09:35:00 SELL STOP at: 446.05 [close]
2024-04-05 09:35:00 SELL at: 436.58 SL at: 437.59 TP at: 426.48
2024-04-05 09:40:00 BUY STOP at: 437.59 [close]
2024-04-08 09:35:00 SELL at: 440.5 SL at: 441.

 95%|█████████▌| 182695/191412 [00:05<00:00, 37189.31it/s]

2024-10-22 09:35:00 SELL at: 492.71 SL at: 493.19 TP at: 487.91
2024-10-22 09:35:00 BUY STOP at: 493.19 [close]
2024-10-23 09:35:00 SELL at: 493.19 SL at: 494.25 TP at: 482.59
BUY at: 488.37 [close]
2024-10-24 09:35:00 SELL at: 491.0 SL at: 492.19 TP at: 479.1
2024-10-24 10:00:00 BUY STOP at: 492.19 [close]
2024-10-25 09:35:00 BUY at: 496.19 SL at: 495.12 TP at: 506.89
2024-10-25 13:40:00 SELL STOP at: 495.12 [close]
2024-10-28 09:35:00 SELL at: 497.51 SL at: 498.52 TP at: 487.41
BUY at: 495.4 [close]
2024-10-29 09:35:00 SELL at: 494.05 SL at: 496.08 TP at: 473.75
2024-10-29 10:00:00 BUY STOP at: 496.08 [close]
2024-10-30 09:35:00 SELL at: 498.37 SL at: 499.6 TP at: 486.07
2024-10-30 10:05:00 BUY STOP at: 499.6 [close]
2024-10-31 09:35:00 SELL at: 490.38 SL at: 492.43 TP at: 469.88
BUY at: 483.9 [close]
2024-11-01 09:35:00 BUY at: 486.54 SL at: 485.2 TP at: 499.94
SELL at: 487.4 [close]
2024-11-04 09:35:00 SELL at: 485.11 SL at: 487.32 TP at: 463.01
2024-11-04 10:00:00 BUY STOP at: 487

100%|██████████| 191412/191412 [00:05<00:00, 35594.07it/s]

2025-05-23 09:35:00 BUY at: 507.62 SL at: 505.58 TP at: 528.02
SELL at: 509.29 [close]
2025-05-27 09:35:00 SELL at: 515.59 SL at: 516.8 TP at: 503.49
2025-05-27 09:55:00 BUY STOP at: 516.8 [close]
2025-05-28 09:35:00 BUY at: 522.71 SL at: 521.85 TP at: 531.31
2025-05-28 09:45:00 SELL STOP at: 521.85 [close]
2025-05-29 09:35:00 SELL at: 524.81 SL at: 526.48 TP at: 508.11
BUY at: 519.96 [close]
2025-05-30 09:35:00 SELL at: 518.48 SL at: 519.66 TP at: 506.68
2025-05-30 10:25:00 BUY STOP at: 519.66 [close]
2025-06-02 09:35:00 BUY at: 520.09 SL at: 517.22 TP at: 548.79
2025-06-02 10:00:00 SELL STOP at: 517.22 [close]
2025-06-03 09:35:00 SELL at: 523.23 SL at: 524.77 TP at: 507.83
2025-06-03 09:45:00 BUY STOP at: 524.77 [close]
2025-06-04 09:35:00 BUY at: 528.73 SL at: 527.84 TP at: 537.63
2025-06-04 10:00:00 SELL STOP at: 527.84 [close]
2025-06-05 09:35:00 SELL at: 529.92 SL at: 530.85 TP at: 520.62
2025-06-05 10:40:00 BUY STOP at: 530.85 [close]
2025-06-06 09:35:00 SELL at: 529.55 SL at: 5

### Test 1 VS Base Strategy VS Buy Hold Intraday

In [15]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = test1.dates, y = test1.account, mode = 'lines', name = 'Test 1'))
fig2.add_trace(go.Scatter(x = base_strategy_dates, y = base_strategy_account, mode = 'lines', name = 'Base Strategy'))
fig2.add_trace(go.Scatter(x = buy_hold_intraday_account.index, y = buy_hold_intraday_account.values, mode = 'lines', name = 'Buy & Hold Intraday'))
fig2.update_layout(title = dict(text = "Test 1 VS Base Strategy VS Buy & Hold Intraday", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### Test 1 Money Management

In [16]:
test2 = trading_strategy_sim()

# test2 = Test 1 Money Management

In [17]:
for x in tqdm(range(0, len(test1_db))):
    # Entry Logic
    if test1_db['time'].iloc[x] == dt.time(9,35,0):
        if test1_db['direction'].iloc[x] == 'Buy':
            test2.buy_pos = 1
            test2.buy_price = test1_db['open'].iloc[x]
            test2.opening_price = test2.buy_price
            test2.stop_loss = test1_db['stop_loss'].iloc[x]
            test2.take_profit = test1_db['target'].iloc[x]
            test2.risk = abs(test2.opening_price - test2.stop_loss)
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'BUY at:', test2.buy_price,
                  'SL at:', test2.stop_loss, 'TP at:', test2.take_profit)

        if test1_db['direction'].iloc[x] == 'Sell':
            test2.sell_pos = 1
            test2.sell_price = test1_db['open'].iloc[x]
            test2.opening_price = test2.sell_price
            test2.stop_loss = test1_db['stop_loss'].iloc[x]
            test2.take_profit = test1_db['target'].iloc[x]
            test2.risk = abs(test2.opening_price - test2.stop_loss)
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'SELL at:', test2.sell_price,
                  'SL at:', test2.stop_loss, 'TP at:', test2.take_profit)

    # Long Position Management 
    if test2.buy_pos == 1 and test2.sell_pos == 0:
        if test1_db['low'].iloc[x] <= test2.stop_loss:
            test2.sell_pos = 1
            test2.sell_price = test2.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL STOP at:', test2.sell_price, '[close]')
        elif test1_db['high'].iloc[x] >= test2.take_profit:
            test2.sell_pos = 1
            test2.sell_price = test2.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL TP at:', test2.sell_price, '[close]')

    # Short Position Management
    if test2.buy_pos == 0 and test2.sell_pos == 1:
        if test1_db['high'].iloc[x] >= test2.stop_loss:
            test2.buy_pos = 1
            test2.buy_price = test2.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY STOP at:', test2.buy_price, '[close]')
        elif test1_db['low'].iloc[x] <= test2.take_profit:
            test2.buy_pos = 1
            test2.buy_price = test2.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY TP at:', test2.buy_price, '[close]')

    # End Day
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if test2.buy_pos == 1 and test2.sell_pos == 0:
            test2.sell_pos = 1
            test2.sell_price = test1_db['close'].iloc[x]
            print('SELL at:', test1_db['close'].iloc[x], '[close]')

        if test2.buy_pos == 0 and test2.sell_pos == 1:
            test2.buy_pos = 1
            test2.buy_price = test1_db['close'].iloc[x]
            print('BUY at:', test1_db['close'].iloc[x], '[close]')

    # Trade Record
    if test2.buy_pos == 1 and test2.sell_pos == 1:
        test2.record_trade(test1_db['date'].iloc[x])
        test2.reset()


  4%|▍         | 7753/191412 [00:00<00:04, 39076.49it/s]

2016-01-04 09:35:00 SELL at: 109.13 SL at: 109.45 TP at: 105.93
2016-01-04 15:55:00 BUY STOP at: 109.45 [close]
2016-01-05 09:35:00 SELL at: 109.85 SL at: 110.18 TP at: 106.55
BUY at: 109.24 [close]
2016-01-06 09:35:00 BUY at: 107.57 SL at: 107.41 TP at: 109.17
SELL at: 108.27 [close]
2016-01-07 09:35:00 SELL at: 105.8 SL at: 106.0 TP at: 103.8
2016-01-07 09:35:00 BUY STOP at: 106.0 [close]
2016-01-08 09:35:00 BUY at: 106.06 SL at: 105.53 TP at: 111.36
2016-01-08 10:10:00 SELL STOP at: 105.53 [close]
2016-01-11 09:35:00 BUY at: 104.88 SL at: 104.73 TP at: 106.38
2016-01-11 09:35:00 SELL STOP at: 104.73 [close]
2016-01-12 09:35:00 BUY at: 105.6 SL at: 105.24 TP at: 109.2
2016-01-12 10:45:00 SELL STOP at: 105.24 [close]
2016-01-13 09:35:00 SELL at: 105.97 SL at: 106.12 TP at: 104.47
2016-01-13 09:35:00 BUY STOP at: 106.12 [close]
2016-01-14 09:35:00 SELL at: 102.17 SL at: 102.44 TP at: 99.47
2016-01-14 10:25:00 BUY STOP at: 102.44 [close]
2016-01-15 09:35:00 BUY at: 101.02 SL at: 100.44 

  6%|▌         | 11661/191412 [00:00<00:04, 37135.89it/s]

2016-06-13 09:35:00 BUY at: 108.53 SL at: 108.32 TP at: 110.63
2016-06-13 09:35:00 SELL STOP at: 108.32 [close]
2016-06-14 09:35:00 BUY at: 108.14 SL at: 107.74 TP at: 112.14
2016-06-14 10:25:00 SELL STOP at: 107.74 [close]
2016-06-15 09:35:00 SELL at: 108.15 SL at: 108.33 TP at: 106.35
2016-06-15 09:40:00 BUY STOP at: 108.33 [close]
2016-06-16 09:35:00 SELL at: 107.14 SL at: 107.27 TP at: 105.84
2016-06-16 12:20:00 BUY STOP at: 107.27 [close]
2016-06-17 09:35:00 SELL at: 107.22 SL at: 107.56 TP at: 103.82
BUY at: 106.52 [close]
2016-06-20 09:35:00 BUY at: 107.78 SL at: 107.72 TP at: 108.38
2016-06-20 14:15:00 SELL STOP at: 107.72 [close]
2016-06-21 09:35:00 BUY at: 107.42 SL at: 107.25 TP at: 109.12
2016-06-21 10:20:00 SELL STOP at: 107.25 [close]
2016-06-22 09:35:00 SELL at: 107.56 SL at: 107.6 TP at: 107.16
2016-06-22 09:35:00 BUY STOP at: 107.6 [close]
2016-06-23 09:35:00 SELL at: 107.9 SL at: 108.12 TP at: 105.7
2016-06-23 10:25:00 BUY STOP at: 108.12 [close]
2016-06-24 09:35:00 B

 14%|█▎        | 25845/191412 [00:00<00:04, 34229.95it/s]

...

2024-08-20 09:35:00 BUY at: 481.48 SL at: 480.33 TP at: 492.98
2024-08-20 10:15:00 SELL STOP at: 480.33 [close]
2024-08-21 09:35:00 SELL at: 480.51 SL at: 481.16 TP at: 474.01
2024-08-21 09:35:00 BUY STOP at: 481.16 [close]
2024-08-22 09:35:00 BUY at: 484.94 SL at: 484.6 TP at: 488.34
2024-08-22 09:35:00 SELL STOP at: 484.6 [close]
2024-08-23 09:35:00 SELL at: 478.16 SL at: 479.42 TP at: 465.56
2024-08-23 10:00:00 BUY STOP at: 479.42 [close]
2024-08-26 09:35:00 BUY at: 480.18 SL at: 478.97 TP at: 492.28
2024-08-26 09:45:00 SELL STOP at: 478.97 [close]
2024-08-27 09:35:00 SELL at: 473.57 SL at: 474.06 TP at: 468.67
2024-08-27 09:50:00 BUY STOP at: 474.06 [close]
2024-08-28 09:35:00 BUY at: 476.35 SL at: 476.17 TP at: 478.15
2024-08-28 09:35:00 SELL STOP at: 476.17 [close]
2024-08-29 09:35:00 BUY at: 475.48 SL at: 472.78 TP at: 502.48
2024-08-29 14:30:00 SELL STOP at: 472.78 [close]
2024-08-30 09:35:00 SELL at: 474.57 SL at: 475.45 TP at: 465.77
2024-08-30 09:45:00 BUY STOP at: 475.45 [c

 96%|█████████▌| 183409/191412 [00:05<00:00, 36771.56it/s]

2025-03-26 09:35:00 SELL at: 492.05 SL at: 492.81 TP at: 484.45
2025-03-26 09:35:00 BUY STOP at: 492.81 [close]
2025-03-27 09:35:00 SELL at: 481.68 SL at: 483.4 TP at: 464.48
2025-03-27 09:50:00 BUY STOP at: 483.4 [close]
2025-03-28 09:35:00 SELL at: 478.7 SL at: 479.9 TP at: 466.7
2025-03-28 09:35:00 BUY STOP at: 479.9 [close]
2025-03-31 09:35:00 SELL at: 458.7 SL at: 462.52 TP at: 420.5
2025-03-31 11:20:00 BUY STOP at: 462.52 [close]
2025-04-01 09:35:00 SELL at: 467.0 SL at: 468.54 TP at: 451.6
2025-04-01 09:45:00 BUY STOP at: 468.54 [close]
2025-04-02 09:35:00 BUY at: 468.27 SL at: 465.86 TP at: 492.37
SELL at: 476.15 [close]
2025-04-03 09:35:00 BUY at: 459.52 SL at: 456.11 TP at: 493.62
2025-04-03 10:00:00 SELL STOP at: 456.11 [close]
2025-04-04 09:35:00 BUY at: 438.31 SL at: 436.71 TP at: 454.31
2025-04-04 09:35:00 SELL STOP at: 436.71 [close]
2025-04-07 09:35:00 SELL at: 408.0 SL at: 410.0 TP at: 388.0
2025-04-07 09:50:00 BUY STOP at: 410.0 [close]
2025-04-08 09:35:00 BUY at: 438

100%|██████████| 191412/191412 [00:05<00:00, 35312.30it/s]

SELL at: 579.86 [close]
2025-08-15 09:35:00 SELL at: 578.55 SL at: 579.84 TP at: 565.65
BUY at: 577.23 [close]
2025-08-18 09:35:00 BUY at: 577.56 SL at: 576.42 TP at: 588.96
2025-08-18 09:50:00 SELL STOP at: 576.42 [close]
2025-08-19 09:35:00 SELL at: 574.1 SL at: 576.56 TP at: 549.5
BUY at: 569.28 [close]
2025-08-20 09:35:00 SELL at: 566.98 SL at: 568.45 TP at: 552.28
BUY at: 565.85 [close]
2025-08-21 09:35:00 SELL at: 563.91 SL at: 564.66 TP at: 556.41
2025-08-21 09:35:00 BUY STOP at: 564.66 [close]
2025-08-22 09:35:00 SELL at: 563.95 SL at: 564.85 TP at: 554.95
2025-08-22 09:35:00 BUY STOP at: 564.85 [close]
2025-08-25 09:35:00 SELL at: 569.76 SL at: 570.92 TP at: 558.16
2025-08-25 09:55:00 BUY STOP at: 570.92 [close]
2025-08-26 09:35:00 BUY at: 570.59 SL at: 569.76 TP at: 578.89
2025-08-26 09:35:00 SELL STOP at: 569.76 [close]
2025-08-27 09:35:00 BUY at: 571.94 SL at: 571.22 TP at: 579.14
2025-08-27 09:40:00 SELL STOP at: 571.22 [close]
2025-08-28 09:35:00 BUY at: 576.2 SL at: 573.

### Buy & Hold Money Management

In [18]:
buy_hold_open1 = qqq_historical_data['open'].iloc[0]
buy_hold_units1 = int(25000/buy_hold_open1)
buy_hold_close1 = qqq_historical_data.groupby(qqq_historical_data['date'])['close'].last()
buy_hold_account1 = 25000 + (buy_hold_close1 - buy_hold_open1)*buy_hold_units1

### Test 1 Money Management VS Buy &  Hold Money Management 

In [19]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x = test2.dates, y = test2.account, mode = 'lines', name = 'Test 1'))
fig4.add_trace(go.Scatter(x = buy_hold_account1.index, y = buy_hold_account1.values, mode = 'lines', name = 'Buy & Hold'))
fig4.update_layout(title = dict(text = "Test 1 MM VS Buy & Hold MM", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

 ### Traded Units Test 1 Money Management

In [20]:
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x = test2.dates, y = test2.unit_list, mode = 'lines', name = 'Test 1'))
fig5.update_layout(title = dict(text = "Traded Units Test 1 MM", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

In [21]:
print("Average Traded Units per Trade:", np.mean(test2.unit_list))

Average Traded Units per Trade: 1373.6687242798355


In [22]:
print("Average Profit per Unit:", np.mean(test2.unit_trades)) 

Average Profit per Unit: 0.0790905349794245


### Test 1 MM + Slippage Simulation

In [23]:
class trading_strategy_sim_slip:
    def __init__(self):
        # --- Account ---
        self.initial_account = 25000
        self.account = [self.initial_account]

        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real, with slippage) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.fee = 0.0005

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.r_history = []
        self.dates = []
        self.unit_trade = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = int(
            min(
                (self.account[-1] * 0.01) / self.risk,
                (4 * self.account[-1]) / self.opening_price
            )
        )
        self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, step=750, slip=0.01):
        remaining = units
        level = 0
        total_cost = 0

        while remaining > 0:
            qty = min(step, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            total_cost += qty * exec_price
            remaining -= qty
            level += 1

        return total_cost / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if direction == 'Buy':
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def record_trade(self, date):
        pnl = (
            (self.real_sell_price - self.real_buy_price) * self.units
            - 2 * self.units * self.fee
        )

        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)

        r = (self.sell_price - self.buy_price) / abs(
            self.opening_price - self.stop_loss
        )

        self.r_history.append(r)
        self.dates.append(date)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)

    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    def restart(self):
        self.__init__

In [24]:
test3 = trading_strategy_sim_slip()

# test3 = Test 1 MM + Slippage Sim

In [26]:
for x in tqdm(range(0, len(test1_db))):

    # ================= ENTRY =================
    if test1_db['time'].iloc[x] == dt.time(9,35,0):

        if test1_db['direction'].iloc[x] == 'Buy':
            test3.buy_pos = 1
            test3.buy_price = test1_db['open'].iloc[x]
            test3.opening_price = test3.buy_price
            test3.stop_loss = test1_db['stop_loss'].iloc[x]
            test3.take_profit = test1_db['target'].iloc[x]
            test3.risk = abs(test3.opening_price - test3.stop_loss)

            test3.apply_entry('Buy')

        if test1_db['direction'].iloc[x] == 'Sell':
            test3.sell_pos = 1
            test3.sell_price = test1_db['open'].iloc[x]
            test3.opening_price = test3.sell_price
            test3.stop_loss = test1_db['stop_loss'].iloc[x]
            test3.take_profit = test1_db['target'].iloc[x]
            test3.risk = abs(test3.opening_price - test3.stop_loss)

            test3.apply_entry('Sell')

    # ================= LONG =================
    if test3.buy_pos == 1 and test3.sell_pos == 0:

        if test1_db['low'].iloc[x] <= test3.stop_loss:
            test3.sell_pos = 1
            test3.sell_price = test3.stop_loss

            test3.apply_stop('Buy')

        elif test1_db['high'].iloc[x] >= test3.take_profit:
            test3.sell_pos = 1
            test3.sell_price = test3.take_profit
            test3.real_sell_price = test3.sell_price  # NO slippage

    # ================= SHORT =================
    if test3.buy_pos == 0 and test3.sell_pos == 1:

        if test1_db['high'].iloc[x] >= test3.stop_loss:
            test3.buy_pos = 1
            test3.buy_price = test3.stop_loss

            test3.apply_stop('Sell')

        elif test1_db['low'].iloc[x] <= test3.take_profit:
            test3.buy_pos = 1
            test3.buy_price = test3.take_profit
            test3.real_buy_price = test3.buy_price  # NO slippage

    # ================= END DAY =================
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if test3.buy_pos == 1 and test3.sell_pos == 0:
            test3.sell_pos = 1
            test3.sell_price = test1_db['close'].iloc[x]
            test3.real_sell_price = test3.sell_price

        if test3.buy_pos == 0 and test3.sell_pos == 1:
            test3.buy_pos = 1
            test3.buy_price = test1_db['close'].iloc[x]
            test3.real_buy_price = test3.buy_price

    # ================= RECORD =================
    if test3.buy_pos == 1 and test3.sell_pos == 1:
        test3.record_trade(test1_db['date'].iloc[x])
        test3.reset()


100%|██████████| 191412/191412 [00:04<00:00, 40616.28it/s]


### Test 1 MM VS Test 1 MM + Slippage Simulation VS Buy & Hold

In [27]:
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x = test2.dates, y = np.cumsum(test2.trades_pnl), mode = 'lines', name = 'Test 1'))
fig6.add_trace(go.Scatter(x = test3.dates, y = np.cumsum(test3.trades_pnl), mode = 'lines', name = 'Test 1 + Slippage'))
fig6.add_trace(go.Scatter(x = buy_hold_account1.index, y = buy_hold_account1.values - 25000, mode = 'lines', name = 'Buy & Hold'))
fig6.update_layout(title = dict(text = "Test 1 MM VS Test 1 MM + Slippage Sim. VS Buy & Hold", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))